In [2]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [3]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:0

In [4]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.0 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
from langchain import OpenAI, VectorDBQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
import os
import openai

In [6]:
import os
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document

In [8]:
# Importing the necessary library for mounting Google Drive
from google.colab import drive

# Mounting Google Drive to the Colab environment
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [9]:
# Reading Fashion dataset
file_path = '/content/drive/MyDrive/Fashion Search AI/Fashion Dataset v2.csv'
fashion_data = pd.read_csv(file_path)
fashion_data.head()


,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,16331376,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,"Kurta, Trousers, Dupatta",4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,14709966,Nayo Women Red Floral Printed Kurta With Trous...,"Kurta, Trouser, Dupatta",3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,11056154,AHIKA Women Black & Green Printed Straight Kurta,Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."


In [10]:
# Reading API Key
base_path = "/content/drive/MyDrive/Fashion Search AI/"


with open(base_path + "API Key.txt", "r", encoding='utf-8-sig') as f:
    api_key = f.read().strip()


openai.api_key = api_key

In [11]:
display(fashion_data)

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,16331376,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,"Kurta, Trousers, Dupatta",4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,14709966,Nayo Women Red Floral Printed Kurta With Trous...,"Kurta, Trouser, Dupatta",3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,11056154,AHIKA Women Black & Green Printed Straight Kurta,Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."
...,...,...,...,...,...,...,...,...,...,...,...
14209,15415116,Flying Machine Women Blue Solid Mock-Collar Fr...,Sweatshirt,2299.0,Blue,Flying Machine,http://assets.myntassets.com/assets/images/154...,13.0,4.076923,Blue solid front-open sweatshirt has a mock co...,"{'Body Shape ID': '443,424,324', 'Body or Garm..."
14210,16470114,Juelle Women Green Printed Hooded Sweatshirt,Hooded Sweatshirt,2299.0,Green,Juelle,http://assets.myntassets.com/assets/images/164...,NaN,NaN,"Green printed sweatshirt has a hooded, 2 pock...","{'Body Shape ID': '443,424,324', 'Body or Garm..."
14211,16382150,Vero Moda Women Pink Sweatshirt,Sweatshirt,2299.0,Pink,Vero Moda,http://assets.myntassets.com/assets/images/163...,NaN,NaN,"Pink solid sweatshirt has a mock collar, 2 ka...","{'Body Shape ID': '443,424,324', 'Body or Garm..."
14212,16379664,Vero Moda Women Blue Sweatshirt,Sweatshirt,2299.0,Blue,Vero Moda,http://assets.myntassets.com/assets/images/163...,NaN,NaN,"Blue solid sweatshirt has a round neck, long s...","{'Body Shape ID': '443,424,324', 'Body or Garm..."


In [13]:
# Ensure required columns exist
required_columns = ["description", "price", "brand", "colour"]
for col in required_columns:
    if col not in fashion_data.columns:
        raise ValueError(f"Missing required column: {col}")

In [14]:
# Convert data into LangChain documents
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=api_key)
documents = [
    Document(
        page_content=f"{row['name']} - {row['description']}. Price: {row['price']} INR. Brand: {row['brand']}. Color: {row['colour']}",
        metadata={"price": row["price"], "brand": row["brand"], "colour": row["colour"]}
    )
    for _, row in fashion_data.iterrows()
]

<ipython-input-14-6e75aa5ff83b>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=api_key)


In [15]:
# Store embeddings persistently in ChromaDB
db_path = "./chroma_fashion_db"
if os.path.exists(db_path):
    vectorstore = Chroma(persist_directory=db_path, embedding_function=embedding_model)
else:
    vectorstore = Chroma.from_documents(documents, embedding_model, persist_directory=db_path)

vectorstore.persist()
retriever = vectorstore.as_retriever()

<ipython-input-15-16daf114047f>:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [16]:

prompt_template = PromptTemplate(
    input_variables=["context", "query"],
    template="""
    You are a fashion AI assistant. Use the following product database to find the best match:

    {context}

    User Query: {query}

    Provide a detailed response with product suggestions.
    """
)

In [18]:
# Set up LLM and LLMChain
from langchain.chains import LLMChain
llm = OpenAI( openai_api_key=api_key, max_tokens=500)
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# Function to search fashion products
def search_fashion(query):
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in relevant_docs])

    response = llm_chain.run(context=context, query=query)
    return response

<ipython-input-18-3a6ba077449f>:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)


In [19]:
# Interactive loop for multiple queries
if __name__ == "__main__":
    print("Fashion Search AI - Type 'exit' to stop.")

    while True:
        query = input("\nEnter your search query: ")
        if query.lower() in ["exit", "quit"]:
            print("Exiting Fashion Search AI.")
            break

        result = search_fashion(query)
        print("\nSearch Result:\n", result)

Fashion Search AI - Type 'exit' to stop.

Enter your search query: Find me a yellow dress under 2000 INR


<ipython-input-18-3a6ba077449f>:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(query)
<ipython-input-18-3a6ba077449f>:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run(context=context, query=query)



Search Result:
 
There are a few options for a yellow dress under 2000 INR that you can choose from:

1. Rajnandini Yellow & Golden Yoke Design Semi-Stitched Dress Material - This dress material is priced at 4330 INR but it is currently on sale for 1999 INR. It includes a yellow and golden kurta fabric, bottom fabric, and printed dupatta with a fringed border. The material is made of silk blend and shantoon, and can be hand-washed. This is a great option for those who prefer to have a customized fit.

2. 20Dresses Blue & Yellow Printed Basic Jumpsuit - This jumpsuit is priced at 3295 INR but it is currently on sale for 1599 INR. It features a blue and yellow print, square neck, sleeveless design, and 2 pockets. The material is made of polyester and can be machine-washed. This is a great option for those looking for a more casual and comfortable outfit.

3. The Chennai Silks Yellow & Orange Embroidered Ready To Wear Lehenga Choli with Dupatta - This ready-to-wear lehenga choli set is p